In [26]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [27]:
load_dotenv()

True

In [28]:
apikey = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=apikey
)
print(client)

In [43]:
def chatbot(message):
    chat_completion_stream = client.chat.completions.create(
        messages=[
            {"role":"user",
             "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

In [44]:
def prompt_user():
    user_input = input("You: ")
    return user_input

In [45]:
def print_user_message(user_message):
    print("You: ", user_message)
    

In [46]:
def stream_chatbot_response(user_message):
    print("Chatbot: ")
    chatbot(user_message)

In [47]:
prompt = prompt_user()
print_user_message(prompt)
stream_chatbot_response(prompt)

You:  
Chatbot: 
World War I, also known as the Great War, was a global conflict that lasted from 1914 to 1918. It involved most of the world's major powers at the time, divided into two opposing alliances: the Allies (including France, Russia, and the United Kingdom) and the Central Powers (including Germany, Austria-Hungary, and the Ottoman Empire).

The war began in July 1914 with the assassination of Archduke Franz Ferdinand of Austria-Hungary by a Serbian nationalist. This event triggered a series of diplomatic and military escalations that eventually led to a full-scale war.

World War I was characterized by trench warfare, with soldiers on both sides fighting from elaborate systems of trenches and enduring harsh conditions. The war involved significant advances in technology and warfare tactics, including the introduction of machine guns, poison gas, and tanks.
The conflict resulted in a tremendous loss of life, with estimates of around 9 million soldiers killed and 21 million w